<a href="https://colab.research.google.com/github/verenalin/deepQ/blob/master/InventoryManagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import random 

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 24 15:59:24 2019

@author: Verena
"""

# Env#

import numpy as np
import pandas as pd
import random 

class env():
    
    def __init__(self, mode, debug, station_history):
        
        print("Creating A inventory Environment...")
        
        self.mode = mode
        self.seed = random.randint(0, 10)
        self.num_days = 29
        self.current_day = 0
        self.inventory_stock_sim = self.generate_stock(mode)
        self.inventory_stock = self.inventory_stock_sim.copy() # Zurücksetzen für jede Episode
        self.old_stock = self.inventory_stock[0]
        self.new_stock = 0
        self.done = False
        self.reward = 0
        self.inventory_moved = 0
        self.debug = debug
        self.actions = [0, 5, 10, 15] 
        self.n_actions = len(self.actions)
        
        #Features: day, old stock, new stock
        self.n_features = 1
        self.game_over = False

        
        if self.debug == True:
            print("Generating inventory Stock: {}".format(self.mode))
            print("inventory Stock: {}".format(self.inventory_stock))
        
    def generate_stock(self, mode): 
        
        # Liste mit 30 täglichen Inventar-Werten
        # mode: linear oder random
        
        inventory_stock = [20]
        
        if mode == "linear": #Inventar nimmt täglich linear um 3 Stück ab.
            for i in range(1, 30):
                inventory_stock.append(inventory_stock[i-1] - 3) 
             
            
        if mode == "random": #Inventar nimmt täglich um 3 Stück +- 5 ab. Bei positiven Werten handelt es sich um Retoure
            for i in range(1, 30):
                inventory_stock.append(inventory_stock[i-1] - 3 + np.random.random_integers(-5, 5))
            
        return inventory_stock
    
    
    def ping_dqn(self, index): #Informationen für DQN
        action = self.actions[index]
        self.update_stock(action)
        
        if self.inventory_stock[self.current_day] > 50: 
          #Wenn Inventar > Limit, Reward = -30 (Strafzahlung) - 0.2*Inventar(Holding Cost) -0.5*Bestellmenge (Transportkosten p.u.) - 2(Fixe Bestellkosten)
          self.reward = -30 - 0.2*self.inventory_stock[self.current_day] - 2
         
        elif self.inventory_stock[self.current_day] < 0:
          #Wenn Inventar < 0, Reward = -30 (Strafzahlung) - 0.2*Inventar(Holding Cost) -0.5*Bestellmenge (Transportkosten p.u.) - 2(Fixe Bestellkosten)
          self.reward = -30 - 0.2*self.inventory_stock[self.current_day] - 2
        
        else:
          self.reward = - 0.2*self.inventory_stock[self.current_day] - 2
          

        if self.current_day != 29:
            self.update_day()
            self.old_stock = self.inventory_stock[self.current_day - 1]
            self.new_stock = self.inventory_stock[self.current_day]
            
        if self.current_day == 29:
            self.done = True
          
        return self.current_day, self.old_stock, self.new_stock, self.reward, self.done

     
    def get_old_stock(self):
        
        return self.old_stock
    
    def update_stock(self, num_inventory):
        
        # update Inventar basierend auf Aktion in t
        if self.current_day != 30:
            for day in range(self.current_day+1, len(self.inventory_stock)):
                self.inventory_stock[day] += num_inventory #Inventar, Nachfrage bereits abgezogen, + Anzahl Orders
                #if day < len(self.inventory_stock)-1:
                    #self.exp_inventory_stock[day] += num_inventory
                
            self.inventory_moved = num_inventory #Anzahl Ordering
        
        else:
            if self.debug == True:
                print("Last day. Cannot Move inventorys.")
            pass
        
        return
    
    def update_day(self):
        
        # update current_day 
        self.current_day += 1
        
        if self.debug == True:
            print("Tick... Forwarded Current day")
                
        return
    
    def reset(self):
        
        if self.debug == True:
            print("Reset Environment ...")
        
        self.num_days = 29
        self.current_day = 0
        self.inventory_stock = self.inventory_stock_sim.copy()
       # self.exp_inventory_stock = self.exp_inventory_stock_sim.copy()
        self.done = False
        self.reward = 0
        self.inventory_moved = 0
        self.old_stock = self.inventory_stock[0]
        self.new_stock = 0
        #self.expected_stock = self.exp_inventory_stock[0]
        #self.expected_stock_new = 0
        #return (self.current_day, self.old_stock, self.new_stock)
        
    def current_stock(self):
        
        return self.inventory_stock[self.current_day]
    
    def get_sim_stock(self):
        
        return self.inventory_stock

In [0]:
# helper, Eingabe für Form der Nachfrage #

def user_input():
    
    '''
    This function creates all initial parameter for the training based on 
    user inputs.
    '''
    
    episode_list = [eps for eps in range(100, 250, 100)]
    
    # ---- inventory Stock Parameters ----
    # linear: a linear increasing inventory stock with 3 additional inventorys per day
    # random: a linear increasing inventory stock with random fluctuation
    # actual_1: randomly pick traffic from one citiinventory stations
    # -------------------------------
    
    data = input("Linear or Random?: ").lower()
    
    brain = input("Enter agent type (dqn): ").lower()
    
    model_based = None
    
    if brain == 'dqn':
        model_based = False #kann hier if gelöscht werden?
    
    #if data == 'actual':
       # station_history = citi_data_processing
       # print(station_history)
        
   # else:
    station_history = None
    
    return episode_list, data, brain, model_based, station_history

In [0]:
import numpy as np
import pandas as pd

class agent():
    
    
    def __init__(self, epsilon, lr, gamma, current_stock, debug, model_based):
        
        print("Created an Agent ...")
        self.actions = [0, 5, 10, 15]
        self.reward = 0
        self.epsilon = epsilon
        self.lr = lr
        self.gamma = gamma
        self.debug = debug
        self.current_stock = current_stock
        self.model_based = model_based
        
        # performance metric
        self.q_table = pd.DataFrame(columns = self.actions, dtype = np.float64)
        self.monthly_action_history = []
        self.monthly_stock_history = []
       
    def choose_action(self, s, ex):
        
        '''
        This funciton choose an action based on Q Table. It also does 
        validation to ensure stock will not be negative after moving inventorys.
        Input: 
            - s: current inventory
            - ex: expected inventory stock in subsequent day (based on random forests prediction)
        
        Output:
            - action: number of inventorys to move
        
        '''
        
        self.check_state_exist(s)
        self.current_stock = s
        #self.expected_stock = ex
        
        # find valid action based on current stock 
        #valid_state_action = self.find_valid_action(self.q_table.loc[s, :])
        if self.model_based == True:
            self.check_state_exist(s)
            valid_state_action = self.q_table.loc[s, :]

        elif self.model_based == False:
            valid_state_action = self.q_table.loc[s, :]
                
        if np.random.uniform() < self.epsilon:
                        
            try:
                # find the action with the highest expected reward
                
                valid_state_action = valid_state_action.reindex(np.random.permutation(valid_state_action.index))
                action = valid_state_action.idxmax()
            
            except:
                # if action list is null, default to 0
                action = 0
                        
            if self.debug == True:
                print("Decided to Move: {}".format(action))
                        
        else:
            
            # randomly choose an action
            # re-pick if the action leads to negative stock
            try:
                action = np.random.choice(valid_state_action.index)
            except:
                action = 0
            
            if self.debug == True:
                print("Randomly Move: {}".format(action))
        
        self.monthly_action_history.append(action)
        self.monthly_stock_history.append(s)
        
        return action
 
    

    def learn(self, s, a, r, s_, ex, g):

        
        '''
        This function updates Q tables after each interaction with the
        environment.
        Input: 
            - s: current inventory stock
            #- ex: expected inventory stock in next day
            - a: current action (number of inventorys to move)
            - r: reward received from current state
            - s_: new inventory stock based on inventory moved and new stock
        Output: None
        '''
        
        if self.debug == True:
            print("Moved inventorys: {}".format(a))
            print("Old inventory Stock: {}".format(s))
            print("New inventory Stock: {}".format(s_))
            print("---")
        
        self.check_state_exist(s_)

        if self.model_based == False:
            q_predict = self.q_table.loc[s, a]
        elif self.model_based == True:
            avg = s
            self.check_state_exist(s)
            q_predict = self.q_table.loc[s, a]
        

        if g == False:
            

            # Updated Q Target Value if it is not end of day  
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()
        
        else:
            # Update Q Target Value as Immediate reward if end of day
            q_target = r

        if self.model_based == False:
            self.q_table.loc[s, a] += self.lr * (q_target - q_predict)
        elif self.model_based == True:
            self.q_table.loc[avg, a] += self.lr * (q_target - q_predict)
        
        return

    
    def check_state_exist(self, state):
        
        # Add a new row with state value as index if not exist
        
        if state not in self.q_table.index:
            
            self.q_table = self.q_table.append(
                pd.Series(
                        [0]*len(self.actions), 
                        index = self.q_table.columns,
                        name = state
                        )
                )
        
        return
    

    def find_valid_action(self, state_action):
        
        '''
        This function check the validity acitons in a given state.
        Input: 
            - state_action: the current state under consideration
        Output:
            - state_action: a pandas Series with only the valid actions that
                            will not cause negative stock
        '''
        
        # remove action that will stock to be negative
        
        for action in self.actions:
            if self.current_stock + action < 0:
                
                if self.debug == True:
                    print("Drop action {}, current stock {}".format(action, self.current_stock))
                
                state_action.drop(index = action, inplace = True)
        
        return state_action
        
    
    def print_q_table(self):
        
        print(self.q_table)


    def get_q_table(self):
        
        return self.q_table

    
    def get_monthly_actions(self):
        
        return self.monthly_action_history
    
    def get_monthly_stocks(self):
        
        return self.monthly_stock_history

    
    def reset_monthly_history(self):
        
        self.monthly_action_history = []
        self.monthly_stock_history = []



In [0]:
# DQN#

np.random.seed(1)
tf.set_random_seed(1)


class DeepQNetwork:
    def __init__(
        self,
        n_actions,
        n_features,
        learning_rate=0.01,
        reward_decay=0.9,
        e_greedy=0.9,
        replace_target_iter=10,
        batch_size=5,
        e_greedy_increment=None,
        output_graph=None
        
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = 100
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max
        self.monthly_stock_history = []
        
        # total learning step
        self.learn_step_counter = 0

        # initialize zero memory [s, a, r, s_]
        self.memory = np.zeros((self.memory_size, n_features*2+2))

        # consist of [target_net, evaluate_net]
        self._build_net()

        t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='eval_net')

        with tf.variable_scope('soft_replacement'):
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        self.sess = tf.Session()

        #if output_graph:
            # $ tensorboard --logdir=logs
         #   tf.summary.FileWriter("logs/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.cost_his = []
        
        
    #changed tf.,,.reulu to sigmoid
    def _build_net(self):
        # ------------------ all inputs ------------------------
        self.s = tf.placeholder(tf.float32, [None, self.n_features], name='s')  # input State
        self.s_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')  # input Next State
        self.r = tf.placeholder(tf.float32, [None, ], name='r')  # input Reward
        self.a = tf.placeholder(tf.int32, [None, ], name='a')  # input Action

        w_initializer, b_initializer = tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)

        # ------------------ build evaluate_net ------------------
        with tf.variable_scope('eval_net', reuse=tf.AUTO_REUSE):
            e1 = tf.layers.dense(self.s, 20, tf.sigmoid, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='e1')
            self.q_eval = tf.layers.dense(e1, self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='q')

        # ------------------ build target_net ------------------
        with tf.variable_scope('target_net', reuse=tf.AUTO_REUSE):
            t1 = tf.layers.dense(self.s_, 20, tf.sigmoid, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='t1')
            self.q_next = tf.layers.dense(t1, self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='t2')

        with tf.variable_scope('q_target'):
            q_target = self.r + self.gamma * tf.reduce_max(self.q_next, axis=1, name='Qmax_s_')    # shape=(None, )
            self.q_target = tf.stop_gradient(q_target)
        with tf.variable_scope('q_eval'):
            a_indices = tf.stack([tf.range(tf.shape(self.a)[0], dtype=tf.int32), self.a], axis=1)
            self.q_eval_wrt_a = tf.gather_nd(params=self.q_eval, indices=a_indices)    # shape=(None, )
        with tf.variable_scope('loss'):
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval_wrt_a, name='TD_error'))
        with tf.variable_scope('train', reuse=tf.AUTO_REUSE):
            self._train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)
    
    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition
        self.memory_counter += 1
    
    def choose_action(self, observation):
        # to have batch dimension when feed into tf placeholder
        #observation = observation[np.newaxis, :]
        #print("hello")
        self.monthly_stock_history.append(observation)
        observation = np.array([[observation]])

        if np.random.uniform() < self.epsilon:
            # forward feed the observation and get q value for every actions
            actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
            #print(">>>>>> action value: {} <<<<<<".format(actions_value))
            action = np.argmax(actions_value)
        else:
            action = np.random.randint(0, self.n_actions)
        return action

        
    def get_monthly_stocks(self):
        
        return self.monthly_stock_history

    
    def reset_monthly_history(self):
        
        # self.monthly_action_history = []
        self.monthly_stock_history = []
    def _replace_target_params():
        pass
      
    def learn(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.target_replace_op)
            print('\ntarget_params_replaced\n')

        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        _, cost = self.sess.run(
            [self._train_op, self.loss],
            feed_dict={
                self.s: batch_memory[:, :self.n_features],
                self.a: batch_memory[:, self.n_features],
                self.r: batch_memory[:, self.n_features + 1],
                self.s_: batch_memory[:, -self.n_features:],
            })

        self.cost_his.append(cost)

        # increasing epsilon
        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1
    

if __name__ == '__main__':
    DQN = DeepQNetwork(2,4, output_graph=False)



In [0]:
# Trainer#




class trainer():
    
    def __init__(self, station_history):
        
        # Session Properties
        self.episodes = []
        self.stock_type = ""
        self.logging = False
        self.env_debug = False
        self.rl_debug = False
        self.inventory_station = None
        self.operator = None
        self.sim_stock = []
        self.model_based = False
        self.method = None
        self.station_history = station_history
        
        # Performance Metric
        self.success_ratio = 0
        self.rewards = []  # [[r from session 1], [r from session 2] ...]
        self.avg_rewards = [] #[np.mean([r from session 1]), np.mean([r from session 2])...]
        self.final_stocks = [] # [[stock from session 1], [stock from session 2] ...]
        self.episode_action_history = []
        self.episode_stock_history = []
        self.session_action_history = []
        self.session_stock_history = []
        self.q_tables = []
        self.actions = [0, 5, 10, 15]
        
    
    def start(self, episodes, stock_type, logging, env_debug, rl_debug, brain, model_based):
        #DQN
        
        self.episodes = episodes
        self.stock_type = stock_type
        self.logging = logging
        self.env_debug = env_debug
        self.rl_debug = rl_debug
        self.brain = brain
        self.model_based = model_based
       
        self.method = 'DQN'
        
        idx = 0
        
        for eps in self.episodes:
        
            # Initiate new evironment and RL agent
            self.inventory_station = env(self.stock_type, debug = self.env_debug, 
                                    station_history = self.station_history)
            self.sim_stock.append(self.inventory_station.get_sim_stock())

            if self.brain == 'dqn':
                self.operator = DeepQNetwork(self.inventory_station.n_actions, self.inventory_station.n_features, 0.01, 0.9)
            else:
                print("Error: pick correct brain")
                break
            
            # Train the RL agent and collect performance stats
            rewards, final_stocks = self.train_operator(idx, len(self.episodes), eps,
            logging = self.logging, brain = self.brain, model_based = self.model_based)
            
            # Log the results from this training session
            self.rewards.append(rewards)
            self.avg_rewards.append(np.mean(rewards))
            self.final_stocks.append(final_stocks)
            #self.q_tables.append(self.operator.get_q_table())
            self.session_action_history.append(self.episode_action_history)
            self.session_stock_history.append(self.episode_stock_history)
            self.reset_episode_history()
            
            # Destroy the environment and agent objects
            self.inventory_station = None
            self.operator = None
            
            idx += 1
        
        if logging == True:
            self.save_session_results_dqn(self.get_timestamp(replace = True))
            
        return
    
    
    def train_operator(self, idx, num_sessions, episodes, logging, brain, model_based):
    
        '''
        This function trains an RL agent by interacting with the inventory station 
        environment. It also tracks and reports performance stats.
        Input:
            - episodes: a int of episode to be trained in this session (e.g. 500)
        Output:
            - reward_list: a list of reward per episode in this sesison
            - final_stocks: a list of final stocks per episode in this session
        '''
        
        print("Start training the Agent ...")
        rewards = 0
        reward_list = []
        final_stocks = []
        step = 0
        
        for eps in range(episodes):
            
            self.inventory_station.reset()
                
            while True:
                
                # Agent picks an action (number of inventorys to move)
                # Agent sends the action to inventory station environment
                # Agent gets feedback from the environment (e.g. reward of the action, new inventory stock after the action, etc.)
                # Agent "learn" the feedback by updating its Q-Table (state, action, reward)
                # Repeat until end of month (30 days)
                # Reset inventory station environment to start a new day, repeat all
                
                
               
                  action = self.operator.choose_action(self.inventory_station.get_old_stock())
                  current_day, old_stock, new_stock, reward, done = self.inventory_station.ping_dqn(action)
                  self.operator.store_transition(old_stock, action, reward, new_stock)
                  if step > 50 and (step % 10 == 0):
                      self.operator.learn()

                #observation_, reward, done = self.inventory_station.ping(action)
            if done == True:
                    
                    print("{} of {} Session | Episode: {} | Final Stock: {} |Final Reward: {:.2f}".format(idx, 
                          num_sessions, eps, old_stock, rewards))

                    
                    reward_list.append(rewards)
                    final_stocks.append(old_stock)
                    rewards = 0
                    
                   
                    self.episode_stock_history.append(self.operator.get_daily_stocks());
                    self.operator.reset_monthly_history()
                                    
                    break


                    step +=1
                    rewards += reward
                
                # Log monthly action history by each episode


            with open('dqn_log.txt', 'a') as f:
                f.write("{} of {} Session | Episode: {} | Final Stock: {} |Final Reward: {:.2f} \n".format(idx, 
                    num_sessions, eps, old_stock, rewards))

                            
        return reward_list, final_stocks
    
    def get_timestamp(self, replace):
        
        if replace == True:
        
            return str(datetime.datetime.now()).replace(" ", "").replace(":", "").\
                        replace(".", "").replace("-", "")
        
        else:
            
            return str(datetime.datetime.now())
    
    
    def reset_episode_history(self):
        
        self.episode_action_history = []
        self.episode_stock_history = []
        
    
    def cal_performance(self):
        
        successful_stocking = []
        
        print("===== Performance =====")
        
        for session in range(len(self.final_stocks)):
            length = len(self.final_stocks[session])
            num_overstock = np.count_nonzero(np.array(self.final_stocks[session]) > 50)
            num_understock = np.count_nonzero(np.array(self.final_stocks[session]) <= 0)
            ratio = (length - num_understock - num_overstock)*100 / length
            
            print("Session {} | Overstock {} Times | Understock {} Times | {}% Successful".format(session, num_overstock, 
                  num_understock, ratio))

            average_reward = round(self.avg_rewards[session], 2)
            print("Average Episode Reward for Session: {}".format(average_reward))
            
            successful_stocking.append(ratio)
        
        return successful_stocking
    
    
    def save_session_results(self, timestamp):
        
        '''
        This function logs the following: 
            - overall success ratio of each session
            - line chart of success ratio by session
            - line chart of reward history by session
            - Q Table of each session
            - Comparison Line Chart of First and Last Episode monthly Actions
        '''
        
        # --- create a session folder ---
        dir_path = "./performance_log/" + timestamp
        
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            
        successful_stocking = self.cal_performance()
        
        # --- Write Success Rate to File ---
        fname = dir_path + "/success_rate - " + timestamp + ".txt"
        
        with open(fname, 'w') as f:
            
            f.write("Logged at {}".format(self.get_timestamp(replace = False)))
            f.write("\n")
            f.write("This training session ran episodes: {}".format(self.episodes))
            f.write("\n")
        
            for session in range(len(successful_stocking)):
                f.write("Session {} | Episodes: {} | Success Rate: {:.2f}%".format(session, 
                        self.episodes[session], successful_stocking[session]))
                f.write("\n")
        
        # --- Plot Overall Success Rate by Episode ---
        
        title = "% of Successful Rebalancing - " + timestamp
        
        fig1 = plt.figure()
        plt.plot(self.episodes, successful_stocking)
        plt.xlabel("Episodes")
        plt.ylabel("% Success Rate")
        plt.title(title)
        fig1.savefig(dir_path + "/session_success_rate_" + timestamp)
        
        # --- Plot Reward History by Training Session ---
        
        for session in range(len(self.rewards)):
            
            fig = plt.figure(figsize=(10, 8))
            
            title = "Reward History by Training Session " + str(session) + " - " + timestamp
            
            x_axis = [x for x in range(self.episodes[session])]
            plt.plot(x_axis, self.rewards[session], label = "Session "+str(session))
            plt.legend()
            plt.xlabel("Episode")
            plt.ylabel("Reward")
            plt.title(title)
            fig.savefig(dir_path + "/reward_history_session_" + \
                        str(session) + timestamp)
            
        # --- Plot Average Reward History by Training Session ---
        figR = plt.figure(figsize=[10, 8])
        lengths = [len(r) for r in self.rewards]
        means = [np.mean(r) for r in self.rewards]
        if len(self.rewards) > 1:
            increment = (lengths[1]-lengths[0])/20
        else:
            increment = lengths[0]/20

        for reward_list in self.rewards:
            Q3 = np.percentile(reward_list, 75)
            Q1 = np.percentile(reward_list, 25)
            M = np.mean(reward_list)
            location = len(reward_list)
            plt.plot([location-increment, location+increment], [Q1, Q1], 'k-')
            plt.plot([location-increment, location+increment], [Q3, Q3], 'k-')
            plt.plot([location, location], [Q1, Q3], 'k-')
            plt.scatter(location, M, s=100, color='dodgerblue')           

        plt.xlabel('Number of Episodes in Session')
        plt.ylabel('Average Reward per Episode')
        plt.title('Average Reward vs. Session Size', size=20)
        plt.xticks(lengths)

        plt.plot(lengths, means, linestyle='--')
        
        figR.savefig(dir_path + "/reward_averages")

        # --- Save Q tables --- 
        
        for session in range(len(self.q_tables)):
            
            self.q_tables[session].to_csv(dir_path + "/q_table_session_" + \
                        str(session) + timestamp + ".csv")
        
        # --- Comparison Line Chart of First and Last Episode for each Session ---
        
        file_path = dir_path + "/action_history"
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)       
        
        
        for session in range(len(self.session_action_history)):
            
            first_eps_idx = 0
            last_eps_idx = len(self.session_action_history[session])-1
            
            fig = plt.figure(figsize=(10, 8))
            title = "Session " + str(session) + " - monthly Action of Eps " + str(first_eps_idx) + " and Eps " + str(last_eps_idx)
            
            x_axis = [x for x in range(len(self.session_action_history[session][0]))]
            plt.plot(x_axis, self.session_action_history[session][0], label = "Eps 0")
            plt.plot(x_axis, self.session_action_history[session][-1], 
                     label = "Eps " + str(last_eps_idx))
            
            plt.legend()
            plt.xlabel("days")
            plt.ylabel("Number of inventorys Moved")
            plt.title(title)
            
            fig.savefig(file_path + "/action_history_" + str(session) + timestamp)
        
        
        # --- Comparison Line Chart of Simulated and Rebalanced inventory Stock --- #
        file_path = dir_path + "/stock_history"
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)       
        
        
        for session in range(len(self.session_stock_history)):
            
            first_eps_idx = 0
            last_eps_idx = len(self.session_action_history[session])-1
            
            fig = plt.figure(figsize=(10, 8))
            title = "[" + self.method + "]" + "Session " + str(session) + " - Original vs. Balanced inventory Stock after " + str(first_eps_idx) + " and Eps " + str(last_eps_idx)
            
            x_axis = [x for x in range(len(self.session_stock_history[session][0]))]
            plt.plot(x_axis, self.sim_stock[session], label = "Original without Balancing")
            plt.plot(x_axis, self.session_stock_history[session][0], label = "Balanced inventory Stock - Eps 0")
            plt.plot(x_axis, self.session_stock_history[session][-1], 
                     label = "Balanced inventory Stock - Eps " + str(last_eps_idx))
            
            plt.axhline(y = 50, c = "r", ls = "--", label = "Upper Stock Limit")
            plt.axhline(y = 0, c = "r", ls = "--", label = "Lower Stock Limit")
            
            plt.legend()
            plt.xlabel("days")
            plt.ylabel("Number of inventory Stock")
            plt.title(title)
            
            fig.savefig(file_path + "/stock_history_" + str(session) + timestamp)
        
        return

    def save_session_results_dqn(self, timestamp):
        dir_path = "./performance_log/" + timestamp
        
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            
        # --- Comparison Line Chart of Simulated and Rebalaned inventory Stock --- #
        file_path = dir_path + "/stock_history"
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)       
        
        
        successful_stocking = self.cal_performance()
        
        # --- Write Success Rate to File ---
        fname = dir_path + "/success_rate - " + timestamp + ".txt"
        
        with open(fname, 'w') as f:
            
            f.write("Logged at {}".format(self.get_timestamp(replace = False)))
            f.write("\n")
            f.write("This training session ran episodes: {}".format(self.episodes))
            f.write("\n")
        
            for session in range(len(successful_stocking)):
                f.write("Session {} | Episodes: {} | Success Rate: {:.2f}%".format(session, 
                        self.episodes[session], successful_stocking[session]))
                f.write("\n")

            # --- Plot Overall Success Rate by Episode ---
        
        title = "% of Successful Rebalancing - " + timestamp
        
        fig1 = plt.figure()
        plt.plot(self.episodes, successful_stocking)
        plt.xlabel("Episodes")
        plt.ylabel("% Success Rate")
        plt.title(title)
        fig1.savefig(dir_path + "/session_success_rate_" + timestamp)

        for session in range(len(self.session_stock_history)):
            
            first_eps_idx = 0
            last_eps_idx = len(self.session_stock_history[session])-1
            
            fig = plt.figure(figsize=(10, 8))
            title = "[" + self.method + "]" + " Session " + str(session) + " - Original vs. Balanced inventory Stock after " + str(first_eps_idx) + " and Eps " + str(last_eps_idx)
            
            x_axis = [x for x in range(len(self.session_stock_history[session][0]))]
            plt.plot(x_axis, self.sim_stock[session], label = "Original without Balancing")
            plt.plot(x_axis, self.session_stock_history[session][0], label = "Balanced inventory Stock - Eps 0")
            plt.plot(x_axis, self.session_stock_history[session][-1], 
                     label = "Balanced inventory Stock - Eps " + str(last_eps_idx))
            
            plt.axhline(y = 50, c = "r", ls = "--", label = "Upper Stock Limit")
            plt.axhline(y = 0, c = "r", ls = "--", label = "Lower Stock Limit")
            
            plt.legend()
            plt.xlabel("days")
            plt.ylabel("Number of inventory Stock")
            plt.title(title)
            
            fig.savefig(file_path + "/stock_history_" + "DQN" + str(session) + timestamp)
        
        return

In [0]:
# main#

if __name__ == "__main__":

    # Get Initial Parameters    
    episode_list, data, brain, model_based, station_history = user_input() 
    trainer_DQN = trainer(station_history)            
    trainer_DQN.start(episode_list, data, logging  = True, env_debug = False, rl_debug = False, brain='dqn', model_based = False)
